In [5]:
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import HuggingFaceEncoder
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import os

from unstructured.partition.md import partition_md

load_dotenv()

def process_mdx_advanced(file_path):
    # partition_md tự động xử lý frontmatter và cấu trúc markdown
    elements = partition_md(filename=file_path)
    
    # Gộp các element lại thành văn bản
    full_text = "\n\n".join([str(el) for el in elements])
    return full_text


In [6]:
encoder = HuggingFaceEncoder(name="nomic-ai/nomic-embed-text-v1.5", score_threshold=0.5 , trust_remote_code=True)

logger.setLevel("WARNING")

splitter = RollingWindowSplitter(
    encoder=encoder,
    min_split_tokens=50,
    max_split_tokens=20000,
    window_size=2,
    plot_splits=True,
    enable_statistics=True
)

pinecone_api = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api)

spec = ServerlessSpec(
    cloud='aws',  # or 'gcp', 'azure'
    region='us-east-1'  # choose appropriate region
)

# Create index if not exists
index_name = 'cp-rag'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # Match your encoder's output dim (e.g., BGE-M3 is 1024? Wait, check encoder)
        metric='cosine',
        spec=spec
    )

# Connect to index
index = pc.Index(index_name)


<All keys matched successfully>


In [7]:
def query(question: str):
    embed = encoder([question])

    query_result = index.query(
        top_k=5,
        vector=embed,
        include_metadata=True,
        include_values=True
    )

    context = ""

    for match in query_result.matches:
        title = match['metadata']['title']
        pre = match['metadata']['prechunk']
        content = match['metadata']['content']
        post = match['metadata']['postchunk']

        context += '\n--------\nTitle: '
        context += title
        context += '\n'
        context += pre
        context += '\n'
        context += content
        context += '\n'
        context += post
        context += '\n'

    print(f'''
You are a extremely helpful assistant in Competittive Programming. 
Use the info in the context to answer the question as much detailed as possible. 
If there are any code, implement it fully and dont hide any line.

If you cannot answer, just say "I don't know"

Context:
{context}

Question:
{question}
    ''')

In [8]:
query("What is Range Query? How to range query with segment tree")


You are a extremely helpful assistant in Competittive Programming. 
Use the info in the context to answer the question as much detailed as possible. 
If there are any code, implement it fully and dont hide any line.

If you cannot answer, just say "I don't know"

Context:

--------
Title: Persistent
int val; Node *l, *r; Node(ll x) : val(x), l(nullptr), r(nullptr) {} Node(Node *ll, Node *rr) : val(0), l(ll), r(rr) {} }; int n, a[100001];     // The initial array and its size Node *roots[100001];  // The persistent array's roots Node *build(int l = 0, int r = n - 1) { if (l == r) return new Node(a[l]); int mid = (l + r) / 2; return new Node(build(l, mid), build(mid + 1, r)); } Node *update(Node *node, int val, int pos, int l = 0, int r = n - 1) { if (l == r) return new Node(val); int mid = (l + r) / 2; if (pos > mid) return new Node(node->l, update(node->r, val, pos, mid + 1, r)); else return new Node(update(node->l, val, pos, l, mid), node->r); } int query(Node *node, int pos, int l =

In [9]:
import ollama

def query_rag(question: str):
    embed = encoder([question]) 
    
    query_result = index.query(
        vector=embed[0] if isinstance(embed[0], list) else embed, 
        top_k=5,
        include_metadata=True,
        include_values=False # Không cần values cho generation, tắt đi cho nhẹ
    )

    context = ""
    for match in query_result.matches:
        meta = match.get('metadata', {})
        title = meta.get('title', 'No Title')
        pre = meta.get('prechunk', '')
        content = meta.get('content', '')
        post = meta.get('postchunk', '')

        context += f'\n--------\nTitle: {title}\n'
        context += f'{pre}\n{content}\n{post}\n'

    print(context)
    # 4. Tạo Prompt hoàn chỉnh
    full_prompt = f'''
    You are an extremely helpful assistant in Competitive Programming. 
    Use the info in the context to answer the question as detailed as possible. 
    If there are any code, implement it fully and dont hide any line.

    If you cannot answer, just say "I don't know"

    Context:
    {context}

    Question:
    {question}
    '''
    
    try:
        response = ollama.chat(
            model='llama3',
            messages=[{'role': 'user', 'content': full_prompt}],
            stream=True
        )
        
        full_content = ""
        for chunk in response:
            content = chunk['message']['content']
            print(content, end='', flush=True)
            full_content += content
        print()
        return full_content
        
    except Exception as e:
        return f"Lỗi khi gọi model: {e}"

    return full_prompt

# --- CHẠY THỬ ---
# Bây giờ hàm đã hoạt động thực tế
answer = query_rag("How to implement Segment Tree?")
print(answer)



--------
Title: Sparse_Segtree
id: sparse-segtree redirects: - /plat/sparse-seg title: Sparse Segment Trees author: Andi Qu prerequisites: - RURQ description: 'Querying big ranges.' frequency: 1 In problems where the query range is at most something like $10^6$, a normal segment tree suffices. However, as soon as we move to bigger ranges ($10^{12}$ in some cases), a normal segment tree results in MLE. Luckily, we can still use a segment tree to solve these types of problems. The main idea is that we don't have to store all the nodes at all times - we create nodes only when needed. In a normal segment tree, an update only affects $\mathcal{O}(\log N)$ nodes, so in a sparse segment tree, we only store $\mathcal{O}(Q \log N)$ nodes! We can implement this efficiently using pointers to a node's children - just like a trie! (Then again, a segment tree is basically a fancier binary trie.) An alternative is to implement the nodes using indices and an array to keep track of each node. This is 

KeyboardInterrupt: 